In [183]:
import sys
import os
import socialsent
from socialsent import util, polarity_induction_methods, seeds
from socialsent.representations.representation_factory import create_representation
from socialsent.polarity_induction_methods import random_walk, _bootstrap_func
import functools
import numpy as np

In [197]:
POSITIVE_DACA = ["racist", "permanent", "heretostay","drumpf", "defenddreamers", "theresistance", "racists","protectdreamers", "bigotry", "undocumented", "supremacist", "supremacists", "future", "stand", "failed", "fail", "taxes", "solidarity", "heretofight"]
NEGATIVE_DACA =["illegal", "illegals", "nodaca", "illegally", "skip",  "maga", "criminal", "criminals", "gang", "rapist", "aliens", "unconstitutional"]
def daca_seeds():
    return POSITIVE_DACA, NEGATIVE_DACA


In [198]:
word_dict = util.load_pickle("twitter_dict.pkl")
print word_dict

Dictionary(14704 unique tokens: [u'militaryparade', u'deferment', u'yellow', u'prices', u'woods']...)


In [199]:
word_dict.doc2idx(["nodaca"])


[108]

In [200]:
word_dict.filter_extremes(no_above = 0.5, no_below=50, keep_tokens = POSITIVE_DACA + NEGATIVE_DACA)
to_keep = sorted(word_dict.dfs, key=lambda w : word_dict.dfs[w], reverse=True)
word_dict.filter_tokens(good_ids=to_keep)
outpath = "data/"


In [201]:
print word_dict
word_dict.doc2idx(["undocumented"])

Dictionary(4954 unique tokens: [u'limited', u'todays', u'chainmigration', u'colleges', u'drumpf']...)


[1009]

In [202]:
sub_vecs = create_representation("SVD", outpath + "vecs")
pos_seeds, neg_seeds = daca_seeds()
sub_vecs = sub_vecs.get_subembed(set(word_dict.token2id.keys()).union(pos_seeds).union(neg_seeds))

In [203]:
def bootstrap(embeddings, positive_seeds, negative_seeds, num_boots=10, score_method=random_walk,
        boot_size=7, return_all=False, n_procs=15, **kwargs):
    map_func = functools.partial(_bootstrap_func, embeddings, positive_seeds, negative_seeds,
            boot_size, score_method, **kwargs)
    polarities_list = [None] * num_boots 
    for boot in range(num_boots):
        print boot
        polarities_list[boot] = map_func(boot)
        #print polarities_list
    if return_all:
        return polarities_list
    else:
        polarities = {}
        for word in polarities_list[0]:
            polarities[word] = np.mean([polarities_list[i][word] for i in range(num_boots)])
        return polarities

In [191]:
polarities_list = bootstrap(sub_vecs, pos_seeds, neg_seeds, return_all=True,
                nn=25, beta=0.9, num_boots=50, n_procs=10)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [205]:
polarities = {}
for word in polarities_list[0]:
    polarities[word] = np.nanmean([polarities_list[i][word] for i in range(50)])



In [206]:
polarities_std = {}
for word in polarities_list[0]:
    polarities_std[word] = np.std([polarities_list[i][word] for i in range(50)])


In [209]:
## add the seeds to polarities dict
for word in POSITIVE_DACA:
    polarities.update({word : 1.0})
    polarities_std.update({word : 0.01})
for word in NEGATIVE_DACA:
    polarities.update({word : 0.0})
    polarities_std.update({word : 0.01})


In [208]:
print len(polarities)
print len(polarities_std)


4766
4766


In [210]:
util.write_pickle(polarities, outpath + "tweets_pol.pkl")

In [211]:
util.write_pickle(polarities_std, outpath + "tweets_pol_sd.pkl")

In [212]:
df = pd.DataFrame(polarities_std.items())
df.to_csv('polarities_std.csv', index=False)